# Transformers

This notebook will focus on the implementation on a customizable model from scratch on the `Tweets` dataset for classification purposes. 

## Without Transformer's Module

We will be following the architecture from the original paper [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf?ref=blog.paperspace.com), with primarily the help of `Keras` and `Tensorflow`.

In [7]:
# Importing all the needed modules
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential, Model

### Transformer Block

This represents one of the blocks in the Transformer architecture. We will the following kinds of layers in this block:

1. We use the `MultiHeadAttention` provided by `Tensorflow` directly to avoid the implementation of the following steps:
- Split the `K`, `V` and `Q` vectors with dimension `embed_dim` into the required vectors based on the number of heads.
- Apply a linear layer to each of them.
- Perform a Scaled Dot Product to calculate the value for each of them.
- Concat all the vectors and then apply a linear classifier on it.
- Apply addition and normalization to the inputs.

1. A `Feed Forward Neural Network` that accepts the number of number of neuros `ff_dim` and the embedding dimension `embed_dim` and genrates a simple `Sequential` layer.

2. A `Normalization` layer to normalize the activations of the previous layer. 
3. A `Dropout` layer to randomly set input units to 0 with a frequency of `rate` at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1 / (1 - rate) such that the sum over all inputs is unchanged. This ensures that the layer stays normalized.

We do not apply a `Masked Multi Attention Head` layer in our implementation as we are going to be using this transformer for classification and not for generation. Thus we do not need to mask the next words in our input. Also due to the nature of the classification task, we do not need seperate `Encoder` and `Decoder` blocks, as now the architecture of both of them would overlap greatly.

In [8]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attentionLayer = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim
        )
        self.feedForwardNN = Sequential(
            [
                Dense(ff_dim, activation="relu"),
                Dense(embed_dim),
            ]
        )
        self.normalization1 = LayerNormalization(epsilon=1e-6)
        self.normalization2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    # inputs: Matrix of the appropiate size
    # training: Boolean to represent the model is being used in training or prediction
    def call(self, inputs, training):
        # Query = Value = Key = inputs
        # Key = Value is the most common case
        attn_output = self.attentionLayer(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        # out1 is the initial result after applying the attention layer and needs to be concatenated with the FFN
        out1 = self.normalization1(inputs + attn_output)

        ffn_output = self.feedForwardNN(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.normalization2(out1 + ffn_output)

### Embedding Blocks

Using this block, we would create two embedding layers, namely for the `tokens` and the `token index positions`. 
1. In the first layer, we initialize the token embeddings from the `vocab` to a space of `embed_dim` dimensions.
2. In the next layer, we intiliaze the positional embeddings. The input dimension would obviously be the length of the sequence and the output dimension would be same as `embed_dim`.

For this layer, it is assumed that all the inputs have been padded to the fixed length `maxlen`.

In [9]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        # positions = [0, 1, 2, ....., maxlen -1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)

        inputs = self.token_emb(inputs)
        return inputs + positions

### Preparing Data and Building the Model

The following code does the following:
- Load the dataset and split it into a `1:3` testing-training dataset.
- Map all the words to a distinct index (from `1` to `vocab_size`) 
- Pad all the tweets to `maxlen` to make them of the same length. 

In the next block, we put together the model with the following layers:
- Embedding Layer (Word + Positional)
- Transformer Block (MultiHead Attention + Feed Forward Neural Network)
- `AveragePooling` layer
- `Dropout` layer
- `Dense` Layer for classification followed by another `Dropout` layer

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv("data/sample_tweets.csv", sep=",", names=["label", "text"], header=0)
tweets = df["text"].values
y = df["label"].values

# Divide into a 25% test set and 75% training set
tweets_train, tweets_test, y_train, y_test = train_test_split(
    tweets, y, test_size=0.25, random_state=1000
)


tokenizer = Tokenizer(num_words=2500)
tokenizer.fit_on_texts(tweets_train)

X_train = tokenizer.texts_to_sequences(tweets_train)
X_test = tokenizer.texts_to_sequences(tweets_test)

vocab_size = (
    len(tokenizer.word_index) + 1
)  # Adding 1 because of reserved 0 index for padding


maxlen = 100
X_train = pad_sequences(X_train, padding="post", maxlen=maxlen)
X_test = pad_sequences(X_test, padding="post", maxlen=maxlen)

In [11]:
embed_dim = 32      # Embedding size for each token
num_heads = 2       # Number of attention heads
ff_dim = 32         # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(
    maxlen, 
    vocab_size, 
    embed_dim
)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(
    embed_dim, 
    num_heads, 
    ff_dim
)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(2, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

### Compilation and Evaluation of the Model

In [12]:
# Compile and train the model
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=50,
    validation_data=(X_test, y_test),
)

# Save the weights of the model
model.save_weights("data/training_checkpoints/transformer1.h5")

# Evaluate the model on the testing data and print the results
print("\n\n\n")
results = model.evaluate(X_test, y_test, verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

Epoch 1/50
6/6 [==============================] - 1s 48ms/step - loss: 0.7461 - accuracy: 0.5173 - val_loss: 0.5840 - val_accuracy: 0.7200
Epoch 2/50
6/6 [==============================] - 0s 20ms/step - loss: 0.6524 - accuracy: 0.6667 - val_loss: 0.5754 - val_accuracy: 0.7200
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 0.6404 - accuracy: 0.6507 - val_loss: 0.6102 - val_accuracy: 0.7200
Epoch 4/50
6/6 [==============================] - 0s 21ms/step - loss: 0.6085 - accuracy: 0.6773 - val_loss: 0.5699 - val_accuracy: 0.7200
Epoch 5/50
6/6 [==============================] - 0s 20ms/step - loss: 0.5790 - accuracy: 0.6853 - val_loss: 0.5439 - val_accuracy: 0.7200
Epoch 6/50
6/6 [==============================] - 0s 21ms/step - loss: 0.5445 - accuracy: 0.7013 - val_loss: 0.5371 - val_accuracy: 0.7280
Epoch 7/50
6/6 [==============================] - 0s 23ms/step - loss: 0.5305 - accuracy: 0.7573 - val_loss: 0.5160 - val_accuracy: 0.7280
Epoch 8/50
6/6 [===========